# Using Python  access SSBD OMERO and IDR for searching meta-data

Updated: 2019-04-02

## Loading json library allows iPython to deal with JSON formatted data

In [1]:
import json

In [2]:
import requests

In [3]:
SSBD_OMERO_URL = "http://ssbd.qbic.riken.jp/image"

In [4]:
IMAGE_ID=1

## Testing some usage of SSBD-OMERO API

In [5]:
image_url = "{base}/webclient/?show=image-{image_id}"
image_viewer = "{base}/webclient/img_detail/{image_id}/"
thumbnail_url = "{base}/webclient/render_thumbnail/{image_id}/"
attributes_url = "{base}/webclient/api/annotations/?type=map&image={image_id}"

print('Image link:', image_url.format(**{'base': SSBD_OMERO_URL, 'image_id': IMAGE_ID}))
print('Image viewer link:', image_viewer.format(**{'base': SSBD_OMERO_URL, 'image_id': IMAGE_ID}))
print('Thumbnail URL:', thumbnail_url.format(**{'base': SSBD_OMERO_URL, 'image_id': IMAGE_ID}))
print('Attributes URL:', attributes_url.format(**{'base': SSBD_OMERO_URL, 'image_id': IMAGE_ID}))

Image link: http://ssbd.qbic.riken.jp/image/webclient/?show=image-1
Image viewer link: http://ssbd.qbic.riken.jp/image/webclient/img_detail/1/
Thumbnail URL: http://ssbd.qbic.riken.jp/image/webclient/render_thumbnail/1/
Attributes URL: http://ssbd.qbic.riken.jp/image/webclient/api/annotations/?type=map&image=1


In [6]:
ANNOTATE_URL = "{base}/webclient/api/annotations/{image_id}/"


In [7]:
qs = {'base': SSBD_OMERO_URL, 'image_id': IMAGE_ID}

url = attributes_url.format(**qs)
r = requests.session().get(url)


if r.status_code == 200:
    metadata = r.json()
    print("\n", metadata)
    print("\n", url)


 {'annotations': [{'description': None, 'class': 'MapAnnotationI', 'date': '2017-09-20T21:51:36Z', 'link': {'owner': {'id': 2}, 'date': '2017-09-20T21:51:36Z', 'id': 36970, 'parent': {'id': 1, 'name': 'img0000.ipm', 'class': 'ImageI'}, 'permissions': {'canAnnotate': False, 'canEdit': False, 'canDelete': False, 'canLink': False}}, 'owner': {'id': 2}, 'values': [['License', 'CC BY-NC-SA'], ['Contactname', 'Shuichi Onami'], ['Organization', 'RIKEN'], ['Department', 'Quantitative Biology Center'], ['Laboratory', 'Laboratory for Developmental Dynamics'], ['Contributers', 'Eru Adachi, Eriko Masuda, Yoko Nagai, Yoko Suzuki, Taeko Oguro, Koji Kyoda, Shuichi Onami'], ['Organism', 'C. elegans'], ['PubMed', 'https://www.ncbi.nlm.nih.gov/pubmed/23172286']], 'ns': 'openmicroscopy.org/omero/client/mapAnnotation', 'id': 37442, 'permissions': {'canAnnotate': False, 'canEdit': False, 'canDelete': False, 'canLink': False}}], 'experimenters': [{'lastName': 'data', 'omeName': 'public_data', 'id': 2, 'fir

## Summary of all the APIs that will be used

* Summary of API for SSBD-OMERO
        * List all the projects in OMERO
                http://ssbd.qbic.riken.jp/image/webgateway/proj/list/
        * List the meta data of a project
                http://ssbd.qbic.riken.jp/image/webclient/api/annotations/?type=map&project=2
        * List all the dataset in a project
                http://ssbd.qbic.riken.jp/image/webgateway/proj/1/children/
        * List all the images in a dataset
                http://ssbd.qbic.riken.jp/image/webgateway/dataset/1/children/
        * List the attributes of an image
                http://ssbd.qbic.riken.jp/image/webclient/api/annotations/?type=map&image=25021
        * Image link:
                http://ssbd.qbic.riken.jp/image/webclient/?show=image-25021
        * Image viewer link:
                http://ssbd.qbic.riken.jp/image/webclient/img_detail/25021/
        * Thumbnail:
                http://ssbd.qbic.riken.jp/image/webclient/render_thumbnail/25021/

## Reference links to OMERO APIs

https://idr.openmicroscopy.org/about/api.html

https://docs.openmicroscopy.org/omero/5.4.0/developers/json-api.html

https://docs.openmicroscopy.org/omero/5.4.10/developers/Web/WebGateway.html?highlight=webgateway


# Pilot code to test the concept of combining the search of meta data in both SSBD-OMERO and in IDR.

As far as I understand there is no direct API that can search for annotation. 

In order to do the search, a list of projects is retrieved. By iterating through the list of projects, annotation data are retrieved and a list of key value pairs is created. Search is then done by searching through the key, value pairs. It is possible to extend this method to search for annotations within the datasets. 

For a server application, the list of projects and attributes can be prefetched once a day or once every update. 

The main disadvantage of this approach is that it is not scalable when the list of projects and their attributes become large. 

## function get_project() returns annotation data as a dictionary given base URL and a project ID

In [8]:
def get_project(base_url,projectid):
    qs = {'base': base_url, 'project': projectid}
    dataset_attributes_url = "{base}/webclient/api/annotations/?type=map&project={project}"
    url = dataset_attributes_url.format(**qs)
    r = requests.session().get(url)

    if r.status_code == 200:
        metadata = r.json()
        data = {}
        for a in metadata['annotations']:
            namespace = a['ns']
            for v in a['values']:
                data[v[0]] = v[1]
        return(data)
           
    else:
        print("error")

In [9]:
result = get_project(SSBD_OMERO_URL, 1)
print(result)

{'License': 'CC BY-NC-SA', 'Contactname': 'Shuichi Onami', 'Organization': 'RIKEN', 'Department': 'Quantitative Biology Center', 'Laboratory': 'Laboratory for Developmental Dynamics', 'Contributers': 'Eru Adachi, Eriko Masuda, Yoko Nagai, Yoko Suzuki, Taeko Oguro, Koji Kyoda, Shuichi Onami', 'Organism': 'C. elegans', 'PubMed': 'https://www.ncbi.nlm.nih.gov/pubmed/23172286'}


In [10]:
result

{'License': 'CC BY-NC-SA',
 'Contactname': 'Shuichi Onami',
 'Organization': 'RIKEN',
 'Department': 'Quantitative Biology Center',
 'Laboratory': 'Laboratory for Developmental Dynamics',
 'Contributers': 'Eru Adachi, Eriko Masuda, Yoko Nagai, Yoko Suzuki, Taeko Oguro, Koji Kyoda, Shuichi Onami',
 'Organism': 'C. elegans',
 'PubMed': 'https://www.ncbi.nlm.nih.gov/pubmed/23172286'}

## Searching for values on all the keys to match koji

In [11]:
[value for key, value in result.items() if 'koji' in value.lower()]

['Eru Adachi, Eriko Masuda, Yoko Nagai, Yoko Suzuki, Taeko Oguro, Koji Kyoda, Shuichi Onami']

## function get_all_projects() return a list of projects

In [12]:
def get_all_projects(base_url): # http://ssbd.qbic.riken.jp/image/webgateway/proj/list/
    qs = {'base': base_url, 'type': 'proj'}
    dataset_attributes_url = "{base}/webgateway/{type}/list"
    url = dataset_attributes_url.format(**qs)
    # print(url)
    r = requests.session().get(url)
    if r.status_code == 200:
        metadata = r.json()
        project_id_list = []
        for a in metadata:
            project_id_list.append(a)
        return(project_id_list)
    else:
        print("error")    


## Getting all the projects in SSBD-OMERO

In [13]:
ssbd_list = get_all_projects(SSBD_OMERO_URL)
ssbd_list

[{'description': 'See details in Inoue and Kondo (2016) Scientific Reports, 6, 33689.',
  'id': 68,
  'name': '38-Inoue-ShellStruct'},
 {'description': 'See details in Takayama and Onami. (2016) Cell Reports, 15(3), 625-637',
  'id': 11,
  'name': '12-Takayama-Oocyte'},
 {'description': 'See details in Kyoda et al. (2013) Nucleic Acids Res 41, D732-D737',
  'id': 1,
  'name': '2-Kyoda-WormEmbryoRNAi'},
 {'description': 'See details in Azuma and Onami (2017) BMC Bioinformatics, 18(1): 307.',
  'id': 101,
  'name': '20-Azuma-WormMembrane'},
 {'description': 'See details in Toyoshima et al. (2016) PLoS Computational Biology, 12(6): e1004970',
  'id': 4,
  'name': '11-Toyoshima-NeuAct'},
 {'description': 'See details in Arata et al. (2016) Cell Reports, 16, 1-13.',
  'id': 67,
  'name': '37-Arata-MolDynPar2'},
 {'description': 'See details in Matsuda et al. (2012) Sci. Signal., 5, ra31.',
  'id': 61,
  'name': '31-Matsuda-SigDyn'},
 {'description': 'See details in Takai et al. (2015) PNAS,

## function get_all_proj_attrib() returns a list of attributes given a URL and a project list

In [14]:
def get_all_proj_attrib(base_url, project_list):
    dataset_attributes_url = "{base}/webclient/api/annotations/?type=map&project={proj}"
    proj_attrib = []
    for proj in project_list:
        qs = {'base': base_url, 'proj': proj['id']}
        url = dataset_attributes_url.format(**qs)
        print(url)
        r = requests.session().get(url)
        if r.status_code == 200:
            metadata = r.json()
        else:
            quit()
        attribute = proj
        for a in metadata['annotations']:
            for v in a['values']:
                attribute[v[0]] = v[1]
        proj_attrib.append(attribute)
    return(proj_attrib)
    


## Get all the attributes of all the projects in SSBD

In [15]:
ssbd_attrib = get_all_proj_attrib(SSBD_OMERO_URL, ssbd_list)

http://ssbd.qbic.riken.jp/image/webclient/api/annotations/?type=map&project=68
http://ssbd.qbic.riken.jp/image/webclient/api/annotations/?type=map&project=11
http://ssbd.qbic.riken.jp/image/webclient/api/annotations/?type=map&project=1
http://ssbd.qbic.riken.jp/image/webclient/api/annotations/?type=map&project=101
http://ssbd.qbic.riken.jp/image/webclient/api/annotations/?type=map&project=4
http://ssbd.qbic.riken.jp/image/webclient/api/annotations/?type=map&project=67
http://ssbd.qbic.riken.jp/image/webclient/api/annotations/?type=map&project=61
http://ssbd.qbic.riken.jp/image/webclient/api/annotations/?type=map&project=71
http://ssbd.qbic.riken.jp/image/webclient/api/annotations/?type=map&project=64
http://ssbd.qbic.riken.jp/image/webclient/api/annotations/?type=map&project=62
http://ssbd.qbic.riken.jp/image/webclient/api/annotations/?type=map&project=52
http://ssbd.qbic.riken.jp/image/webclient/api/annotations/?type=map&project=51
http://ssbd.qbic.riken.jp/image/webclient/api/annotat

In [16]:
ssbd_attrib

[{'description': 'See details in Inoue and Kondo (2016) Scientific Reports, 6, 33689.',
  'id': 68,
  'name': '38-Inoue-ShellStruct',
  'License': 'CC BY',
  'Contactname': 'Shigeru Kondo',
  'Organization': 'Osaka University',
  'Department': 'Graduate School of Frontier Bioscience',
  'Laboratory': 'Laboratory of Pattern Formation',
  'Contributers': 'Shinya Inoue, Shigeru Kondo',
  'Organism': 'Damesites cf. damesi',
  'PubMed': 'http://www.ncbi.nlm.nih.gov/pubmed/27640361'},
 {'description': 'See details in Takayama and Onami. (2016) Cell Reports, 15(3), 625-637',
  'id': 11,
  'name': '12-Takayama-Oocyte',
  'License': 'CC BY-NC-SA',
  'Contactname': 'Shuichi Onami',
  'Organization': 'RIKEN',
  'Department': 'Quantitative Biology Center',
  'Laboratory': 'Laboratory for Developmental Dynamics',
  'Contributers': 'Jun Takayama, Shuichi Onami',
  'Organism': 'C. elegans',
  'PubMed': 'https://www.ncbi.nlm.nih.gov/pubmed/27068469'},
 {'description': 'See details in Kyoda et al. (201

## function search_proj() returns a list of project with id, key, value given a list if attributes and a search term

In [17]:
def search_proj(proj_attrib, term):
    result = []
    for proj in proj_attrib:
        for key, value in proj.items():
            if key != 'id':
                if term in value.lower():
#                    print("project_id:", proj['id'], key, ":", value)
                    result.append({'id': proj['id'], key : value})
    return(result)

## searching SSBD-OMERO for the phrase "elegans"

In [18]:
ssbd_result = search_proj(ssbd_attrib, 'elegans')

In [19]:
ssbd_result

[{'id': 11, 'Organism': 'C. elegans'},
 {'id': 1, 'Organism': 'C. elegans'},
 {'id': 101, 'Organism': 'Caenorhabditis elegans'},
 {'id': 4, 'Organism': 'C. elegans'},
 {'id': 67, 'Organism': 'Caenorhabditis elegans'}]

## searching SSBD-OMERO for the phrase "koji"

In [20]:
search_proj(ssbd_attrib, 'koji')

[{'id': 1,
  'Contributers': 'Eru Adachi, Eriko Masuda, Yoko Nagai, Yoko Suzuki, Taeko Oguro, Koji Kyoda, Shuichi Onami'},
 {'id': 9,
  'Contributers': 'Md. Khayrul Bashar, Koji Komatsu, Toshihiko Fujimori, Tetsuya J. Kobayashi'}]

# Searching IDR

In [22]:
IDR_URL = IDR_URL = "https://idr.openmicroscopy.org"

## Get all the projects from IDR

In [23]:
idr_project_list = get_all_projects(IDR_URL)
idr_project_list

[{'description': 'Publication Title\nHuman lymphoid organ cDC2 and macrophages play complementary roles in the induction of T follicular helper responses\n\nExperiment Description\nImaging mass cytometry of tonsil sections',
  'id': 701,
  'name': 'idr0054-segura-tonsilhyperion/experimentA'},
 {'description': 'Publication Title\nUnbiased pattern analysis reveals highly diverse responses of cytoskeletal systems to cyclic straining\n\nExperiment Description\nCharacterization of cytoskeletal systems (actin, microtubules, vimentin networks) to cyclic straining using confocal laser scanning microscopy, including experiments with nocodazole to depolymerize microtubules.',
  'id': 505,
  'name': 'idr0050-springer-cytoskeletalsystems/experimentA'},
 {'description': 'Publication Title\nA microscopy data set of discrete spatial and temporal single molecule RNA expression in single cells\n\nExperiment Description\nWe performed single molecule in-situ hybridization on yeast cells that have been ex

## Get all the project attributes from IDR

In [24]:
idr_attrib = get_all_proj_attrib(IDR_URL, idr_project_list)

https://idr.openmicroscopy.org/webclient/api/annotations/?type=map&project=701
https://idr.openmicroscopy.org/webclient/api/annotations/?type=map&project=505
https://idr.openmicroscopy.org/webclient/api/annotations/?type=map&project=503
https://idr.openmicroscopy.org/webclient/api/annotations/?type=map&project=504
https://idr.openmicroscopy.org/webclient/api/annotations/?type=map&project=502
https://idr.openmicroscopy.org/webclient/api/annotations/?type=map&project=552
https://idr.openmicroscopy.org/webclient/api/annotations/?type=map&project=501
https://idr.openmicroscopy.org/webclient/api/annotations/?type=map&project=405
https://idr.openmicroscopy.org/webclient/api/annotations/?type=map&project=353
https://idr.openmicroscopy.org/webclient/api/annotations/?type=map&project=151
https://idr.openmicroscopy.org/webclient/api/annotations/?type=map&project=401
https://idr.openmicroscopy.org/webclient/api/annotations/?type=map&project=404
https://idr.openmicroscopy.org/webclient/api/annotat

In [25]:
idr_attrib

[{'description': 'Publication Title\nHuman lymphoid organ cDC2 and macrophages play complementary roles in the induction of T follicular helper responses\n\nExperiment Description\nImaging mass cytometry of tonsil sections',
  'id': 701,
  'name': 'idr0054-segura-tonsilhyperion/experimentA',
  'Study Type': 'imaging mass cytometry',
  'Organism': 'Homo sapiens',
  'Imaging Method': 'imaging mass cytometry',
  'Publication Title': 'Human lymphoid organ cDC2 and macrophages play complementary roles in the induction of T follicular helper responses',
  'Publication Authors': 'Mélanie Durand, Thomas Walter, Tiphène Pirnay, Thomas Naessens, Paul Gueguen, Christel Goudot, Sonia Lameiras, Qing Chang, Nafiseh Talaei, Olga Ornatsky, Tatiana Vassilevskaia, Sylvain Baulande, Sebastian Amigorena, Elodie Segura',
  'License': 'CC-BY 4.0 https://creativecommons.org/licenses/by/4.0/',
  'Copyright': 'Durand et al.',
  'Data Publisher': 'University of Dundee',
  'BioStudies Accession': 'S-BSST221 http

## searching IDR for the phrase "mus"

In [26]:
search_proj(idr_attrib, 'mus')

[{'id': 502, 'Organism': 'Mus musculus'},
 {'id': 405, 'Organism': 'Mus musculus x Mus spretus'},
 {'id': 353, 'Organism': 'Mus musculus'},
 {'id': 352, 'Organism': 'Mus musculus'},
 {'id': 351, 'Organism': 'Mus musculus'},
 {'id': 101, 'Organism': 'Mus musculus'},
 {'id': 301, 'Organism': 'Mus musculus'}]

## function combine_search() returns the projects that matches the search term in both SSBD-OMERO and IDR

In [27]:
def combine_search(term):
    idr_result = search_proj(idr_attrib, term)
    ssbd_result = search_proj(ssbd_attrib, term)
    print("IDR searh results:")
    for i in idr_result:
        for key, value in i.items():
            print("    ", key, ":", str(value), end=' ')
        print()
    print("SSBD searh results:")
    for j in ssbd_result:
        for key, value in j.items():
            print("    ", key, ":", str(value), end=' ')
        print()
    return({'idr': idr_result, 'ssbd': ssbd_result})


## Searching for the term "mus" in both SSBD-OMERO and IDR

In [28]:
all_result = combine_search('mus')

IDR searh results:
     id : 502      Organism : Mus musculus 
     id : 405      Organism : Mus musculus x Mus spretus 
     id : 353      Organism : Mus musculus 
     id : 352      Organism : Mus musculus 
     id : 351      Organism : Mus musculus 
     id : 101      Organism : Mus musculus 
     id : 301      Organism : Mus musculus 
SSBD searh results:
     id : 59      Organism : Mus musculus 
     id : 9      Organism : M. musculus 
     id : 106      Organism : Mus musculus 
     id : 105      Organism : Mus musculus 
     id : 104      Organism : Mus musculus 
     id : 70      Organism : Mus musculus 
     id : 12      Organism : M. musculus 
     id : 3      Organism : M. musculus 
     id : 55      Organism : Mus musculus 
     id : 103      Organism : Mus musculus 
     id : 56      Organism : Mus musculus 
     id : 57      Organism : Mus musculus 
     id : 58      Organism : Mus musculus 
     id : 7      Organism : M. musculus 
     id : 152      Organism : M. musculu

In [29]:
all_result

{'idr': [{'id': 502, 'Organism': 'Mus musculus'},
  {'id': 405, 'Organism': 'Mus musculus x Mus spretus'},
  {'id': 353, 'Organism': 'Mus musculus'},
  {'id': 352, 'Organism': 'Mus musculus'},
  {'id': 351, 'Organism': 'Mus musculus'},
  {'id': 101, 'Organism': 'Mus musculus'},
  {'id': 301, 'Organism': 'Mus musculus'}],
 'ssbd': [{'id': 59, 'Organism': 'Mus musculus'},
  {'id': 9, 'Organism': 'M. musculus'},
  {'id': 106, 'Organism': 'Mus musculus'},
  {'id': 105, 'Organism': 'Mus musculus'},
  {'id': 104, 'Organism': 'Mus musculus'},
  {'id': 70, 'Organism': 'Mus musculus'},
  {'id': 12, 'Organism': 'M. musculus'},
  {'id': 3, 'Organism': 'M. musculus'},
  {'id': 55, 'Organism': 'Mus musculus'},
  {'id': 103, 'Organism': 'Mus musculus'},
  {'id': 56, 'Organism': 'Mus musculus'},
  {'id': 57, 'Organism': 'Mus musculus'},
  {'id': 58, 'Organism': 'Mus musculus'},
  {'id': 7, 'Organism': 'M. musculus'},
  {'id': 152, 'Organism': 'M. musculus'},
  {'id': 154, 'Organism': 'M. musculus'},
